Judge by llama3.3

In [6]:
evaluation_prompt = """You will act as a reviewer to score the following returned memory based on their relevance, conciseness and readability.
The fewer and more relevant the key messages are, the higher the score will be.
Here is the question and answer:{question}

The score range is from 1 to 10 of integer, with 1 being the worst and 10 being the best, ranked in order of importance:

1. Relevance of the returned information and its ability to support generating similar responses.
2. Conciseness— the more concise and noise-free the returned content, the better.
3. Readability for humans, maintaining chronological order to preserve memory continuity.

Please refer to the following three examples for guidance:
A system:{short_memory}, score:{short_score}
B system:{base_dialog}, score:{base_score}
C system:{base_paragraph}, score:{base_p_score}

Next, please score the returned memory for the following systems:

D system:{long_memory}

E system:{long_recall_memory}

Please provide the scores in the following format:
```json
{{
    "D":"1~10",
    "E":"1~10"
}}
```"""

In [7]:
import requests
import json
import re

def llm_create(prompt):
    url = "http://localhost:11434/api/generate"
    data = {
        "model": "llama3.3:70b-instruct-q4_K_M",
        "prompt": prompt,
        "stream":False
    }

    response = requests.post(url, json=data)
    return response.json()['response']

def llm_response_handler(response:str):
    """handle llm response format, especially for llama family"""
    try:
        return json.loads(response)
    except:
        response = response.strip()
        try:
            return json.loads(re.search(r"```json(.*?)```", response, re.DOTALL).group(1).strip())
        except:
            try:
                return json.loads(re.search(r"```(.*?)```", response, re.DOTALL).group(1).strip())
            except:
                return response
            
def get_res(question, short_searched, long_searched, long_recall_searched, base_dialog_searched, base_paragraph_searched, short_s, base_s, base_p_s):
    res = llm_create(evaluation_prompt.format(question=question, short_memory=short_searched, 
                                              long_memory=long_searched, long_recall_memory=long_recall_searched, base_dialog=base_dialog_searched, 
                                              base_paragraph=base_paragraph_searched, short_score=short_s, base_score=base_s, base_p_score=base_p_s))
    res_dict = llm_response_handler(res)
    # print(res_dict)
    # print(evaluation_prompt.format(question=question, short_memory=short_searched, long_memory=long_searched, long_recall_memory=long_recall_searched, base_dialog=base_dialog_searched, base_paragraph=base_paragraph_searched))
    return res_dict.get('D'), res_dict.get('E')

gpt-4o-mini

In [5]:
import pandas as pd
gpt_df = pd.read_json("eval_result_gpt-4o-mini.json", lines=True)
question_df = pd.read_json('questions_0205.json', lines=True)
base_df = pd.read_json("eval_result_base.json", lines=True)
base_score = pd.read_json('eval_result_base_score.json', lines=True)

In [35]:
for row in range(500):
    print(f'---process {row+1}---')
    
    qa_pair = f"user:{question_df['question'][row]}, assistant:{question_df['answer'][row]}"
    
    long_searched = gpt_df.loc[row, 'long_mem_result']
    long_recall_searched = gpt_df.loc[row, 'long_mem_recall_result']
    
    short_searched = base_df.loc[row, 'short_mem_result']
    base_dialog_searched = base_df.loc[row, 'base_dialog']
    base_paragraph_searched = base_df.loc[row, 'base_paragraph']
    
    short_s = base_score.loc[row, 'short_score']
    base_s = base_score.loc[row, 'base_dialog_score']
    base_p_s = base_score.loc[row, 'base_paragraph_score']
    
    long_score, long_recall_score= get_res(qa_pair, short_searched, long_searched, long_recall_searched, base_dialog_searched, base_paragraph_searched, short_s, base_s, base_p_s)
    gpt_df.loc[row, 'long_score_by_llama'] = long_score
    gpt_df.loc[row, 'long_recall_score_by_llama'] = long_recall_score

---process 425---


In [36]:
number=500
observe_df = gpt_df[:number]
print(observe_df['long_score_by_llama'].astype(int).sum()/number)
print(observe_df['long_recall_score_by_llama'].astype(int).sum()/number)

6.662
8.586


qwen2.5

In [37]:
import pandas as pd
qwen_df = pd.read_json('eval_result_qwen.json', lines=True)
question_df = pd.read_json('questions_0205.json', lines=True)
base_df = pd.read_json("eval_result_base.json", lines=True)
base_score = pd.read_json('eval_result_base_score.json', lines=True)

In [52]:
for row in [113, 335]:
    print(f'---process {row+1}---')
    
    qa_pair = f"user:{question_df['question'][row]}, assistant:{question_df['answer'][row]}"
    
    long_searched = qwen_df.loc[row, 'long_mem_result']
    long_recall_searched = qwen_df.loc[row, 'long_mem_recall_result']
    
    short_searched = base_df.loc[row, 'short_mem_result']
    base_dialog_searched = base_df.loc[row, 'base_dialog']
    base_paragraph_searched = base_df.loc[row, 'base_paragraph']
    
    short_s = base_score.loc[row, 'short_score']
    base_s = base_score.loc[row, 'base_dialog_score']
    base_p_s = base_score.loc[row, 'base_paragraph_score']
    
    long_score, long_recall_score= get_res(qa_pair, short_searched, long_searched, long_recall_searched, base_dialog_searched, base_paragraph_searched, short_s, base_s, base_p_s)
    qwen_df.loc[row, 'long_score_by_llama'] = long_score
    qwen_df.loc[row, 'long_recall_score_by_llama'] = long_recall_score

---process 114---
---process 336---


In [53]:
number=500
observe_df = qwen_df[:number]
print(observe_df['long_score_by_llama'].astype(int).sum()/number)
print(observe_df['long_recall_score_by_llama'].astype(int).sum()/number)

6.594
8.516


gemma2

In [ ]:
import pandas as pd
gemma_df = pd.read_json('eval_result_gemma.json', lines=True)
question_df = pd.read_json('questions_0205.json', lines=True)
base_df = pd.read_json("eval_result_base.json", lines=True)
base_score = pd.read_json('eval_result_base_score.json', lines=True)

In [ ]:
for row in range(500):
    print(f'---process {row+1}---')
    
    qa_pair = f"user:{question_df['question'][row]}, assistant:{question_df['answer'][row]}"
    
    long_searched = gemma_df.loc[row, 'long_mem_result']
    long_recall_searched = gemma_df.loc[row, 'long_mem_recall_result']
    
    short_searched = base_df.loc[row, 'short_mem_result']
    base_dialog_searched = base_df.loc[row, 'base_dialog']
    base_paragraph_searched = base_df.loc[row, 'base_paragraph']
    
    short_s = base_score.loc[row, 'short_score']
    base_s = base_score.loc[row, 'base_dialog_score']
    base_p_s = base_score.loc[row, 'base_paragraph_score']
    
    long_score, long_recall_score= get_res(qa_pair, short_searched, long_searched, long_recall_searched, base_dialog_searched, base_paragraph_searched, short_s, base_s, base_p_s)
    gemma_df.loc[row, 'long_score_by_llama'] = long_score
    gemma_df.loc[row, 'long_recall_score_by_llama'] = long_recall_score


---process 204---
---process 205---
---process 206---
---process 207---
---process 208---
---process 209---
---process 210---
---process 211---
---process 212---
---process 213---
---process 214---
---process 215---
---process 216---
---process 217---
---process 218---
---process 219---
---process 220---
---process 221---
---process 222---
---process 223---
---process 224---
---process 225---
---process 226---
---process 227---
---process 228---
---process 229---
---process 230---
---process 231---
---process 232---
---process 233---
---process 234---
---process 235---
---process 236---
---process 237---
---process 238---
---process 239---
---process 240---
---process 241---
---process 242---
---process 243---
---process 244---
---process 245---
---process 246---
---process 247---
---process 248---
---process 249---
---process 250---
---process 251---
---process 252---
---process 253---
---process 254---
---process 255---
---process 256---
---process 257---
---process 258---
---process

In [3]:
number=500
observe_df = gemma_df[:number]
print(observe_df['long_score_by_llama'].astype(int).sum()/number)
print(observe_df['long_recall_score_by_llama'].astype(int).sum()/number)

6.528
7.448


llama3.3

In [4]:
import pandas as pd
llama_3_3_df = pd.read_json('eval_result_llama3_3.json', lines=True)
question_df = pd.read_json('questions_0205.json', lines=True)
base_df = pd.read_json("eval_result_base.json", lines=True)
base_score = pd.read_json('eval_result_base_score.json', lines=True)

In [11]:
for row in range(500):
    print(f'---process {row+1}---')
    
    qa_pair = f"user:{question_df['question'][row]}, assistant:{question_df['answer'][row]}"
    
    long_searched = llama_3_3_df.loc[row, 'long_mem_result']
    long_recall_searched = llama_3_3_df.loc[row, 'long_mem_recall_result']
    
    short_searched = base_df.loc[row, 'short_mem_result']
    base_dialog_searched = base_df.loc[row, 'base_dialog']
    base_paragraph_searched = base_df.loc[row, 'base_paragraph']
    
    short_s = base_score.loc[row, 'short_score']
    base_s = base_score.loc[row, 'base_dialog_score']
    base_p_s = base_score.loc[row, 'base_paragraph_score']
    
    long_score, long_recall_score= get_res(qa_pair, short_searched, long_searched, long_recall_searched, base_dialog_searched, base_paragraph_searched, short_s, base_s, base_p_s)
    llama_3_3_df.loc[row, 'long_score_by_llama'] = long_score
    llama_3_3_df.loc[row, 'long_recall_score_by_llama'] = long_recall_score

---process 171---


In [13]:
number=500
observe_df = llama_3_3_df[:number]
print(observe_df['long_score_by_llama'].astype(int).sum()/number)
print(observe_df['long_recall_score_by_llama'].astype(int).sum()/number)

6.424
8.304
